In [ ]:
import numpy as np
import sys
import os
try:
	caffe_root = os.environ['CAFFE_ROOT'] + '/'
except KeyError:
  	raise KeyError("Define CAFFE_ROOT in ~/.bashrc")

sys.path.insert(1, caffe_root+'python/')
import caffe
import cv2
from py_returnCAMmap import py_returnCAMmap
from py_map2jpg import py_map2jpg
import scipy.io

import matplotlib.pyplot as plt
%matplotlib inline

def im2double(im):
	return cv2.normalize(im.astype('float'), None, 0.0, 1.0, cv2.NORM_MINMAX)

## Be aware that since Matlab is 1-indexed and column-major, 
## the usual 4 blob dimensions in Matlab are [width, height, channels, num]

## In python the dimensions are [num, channels, width, height]

model = 'googlenet'
if model == 'alexnet':
	net_weights = 'models/alexnetplusCAM_imagenet.caffemodel'
	net_model = 'models/deploy_alexnetplusCAM_imagenet.prototxt'
	out_layer = 'fc9'
	last_conv = 'conv7'
	crop_size = 227
elif model == 'googlenet':
# 	net_weights = 'models/imagenet_googleletCAM_train_iter_120000.caffemodel'
# 	net_model = 'models/deploy_googlenetCAM.prototxt'
	net_weights = 'models/places_googleletCAM_train_iter_120000.caffemodel'
	net_model = 'models/deploy_googlenetCAM_places205.prototxt'    
	out_layer = 'CAM_fc'
	crop_size = 224
	last_conv = 'CAM_conv'
else:
	raise Exception('This model is not defined')

categoriesFile = ["../dataset/slamData/categoryIndex_places205.csv",
                 "../dataset/slamData/categories_places365.txt",
                 '../dataset/slamData/categories_hybrid1365.txt']

categoriesNames = np.loadtxt(categoriesFile[0],dtype=str,delimiter='\n')


# load CAM model and extract features
net = caffe.Net(net_model, net_weights, caffe.TEST)

transformer = caffe.io.Transformer({'data': net.blobs['data'].data.shape})
transformer.set_transpose('data', (2,0,1))
transformer.set_mean('data', np.load(caffe_root + 'python/caffe/imagenet/ilsvrc_2012_mean.npy').mean(1).mean(1))
#transformer.set_channel_swap('data', (2,1,0))  # the reference model has channels in BGR order instead of RGB

weights_LR = net.params[out_layer][0].data # get the softmax layer of the network
print("Weights",weights_LR.shape)
# shape: [1000, N] N-> depends on the network


def getScoresAndLastConvAct(img):

    # Take center crop.
    center = np.array(img.shape[:2]) / 2.0
    crop = np.tile(center, (1, 2))[0] + np.concatenate([
        -np.array([crop_size, crop_size]) / 2.0,
        np.array([crop_size, crop_size]) / 2.0
    ])
    crop = crop.astype(int)
    input_ = img#[crop[0]:crop[2], crop[1]:crop[3], :]

#     plt.imshow(input_)
#     plt.show()
    # extract conv features
    net.blobs['data'].reshape(*np.asarray([1,3,crop_size,crop_size])) # run only one image
    net.blobs['data'].data[...][0,:,:,:] = transformer.preprocess('data', input_)
    out = net.forward()
    scores1 = out['prob']
    activation_lastconv = net.blobs[last_conv].data
#     print("Act_lastConv", activation_lastconv.shape)
    
    return scores1, activation_lastconv


def getCAM(img,scores1,activation_lastconv1,showActs=False):

    ## Class Activation Mapping

    topNum = 5 # generate heatmap for top X prediction results
    scoresMean = np.mean(scores1, axis=0)
    ascending_order = np.argsort(scoresMean)
    IDX_category = ascending_order[::-1] # [::-1] to sort in descending order

#     IDX_category = [92,98,151,16,170]
#     IDX_category = [92,60,67,79,78]
    
    curCAMmapAll = py_returnCAMmap(activation_lastconv1, weights_LR[IDX_category[:topNum],:])
    print("camALl",curCAMmapAll.shape)

    curResult = im2double(img)

    avgHeatMap = []
    for j in range(topNum):
        # for one image
        curCAMmap_crops = curCAMmapAll[:,:,j]
#         plt.imshow(curCAMmap_crops)
#         plt.colorbar()
#         plt.show()
        curCAMmapLarge_crops = cv2.resize(curCAMmap_crops, (256,256))
        imgWrite = cv2.normalize(curCAMmap_crops.astype('float'), None, 0.0, 255.0, cv2.NORM_MINMAX)
        cv2.imwrite("./bboxgenerator/heatmaps/"+str(j)+".jpg",imgWrite)
        curHeatMap = cv2.resize(im2double(curCAMmapLarge_crops),(256,256)) # this line is not doing much
        curHeatMap = im2double(curHeatMap)

        curHeatMap = py_map2jpg(curHeatMap, None, 'jet')
        curHeatMap = im2double(img)*0.2+im2double(curHeatMap)*0.7
        
        if showActs:
            plt.imshow(curHeatMap)
            plt.colorbar()
            plt.show()
            print(categoriesNames[IDX_category[j]])
        #   cv2.imshow(categories['categories'][IDX_category[j]][0][0], curHeatMap)
        #   cv2.waitKey(0)
            avgHeatMap.append(curHeatMap)

    if showActs:
        plt.imshow(np.mean(np.array(avgHeatMap),axis=0))
        plt.colorbar()
        plt.show()
    return curCAMmapAll

### Sample Image Test 

In [ ]:
### image = cv2.imread('img2.jpg')
image = cv2.imread('../dataset/slamData/oxford/camTests/5-0001882.png')
# image = cv2.imread('../dataset/models/places-cnn/signBoard.png')
print(image.shape)

# image = cv2.resize(image[:810,:,:], (427, 270))[:,75:-75]
image = cv2.resize(image[:810,:,:],(256,256))
cv2.imwrite("./bboxgenerator/heatmaps/"+str(1)+"-raw.jpg",cv2.resize(image,(14,14)))
print(image.shape)
plt.imshow(image)
plt.show()
scores, activation_lastconv = getScoresAndLastConvAct(image)
camFeats = getCAM(image,scores,activation_lastconv,showActs=True)


### Check weight patterns for similar classes  

In [ ]:
classInds = [92,60,151,120,135]
weightsForClasses = weights_LR[classInds,:]
topUnits = np.argsort(weightsForClasses,axis=1)[:,-20:]
print(categoriesNames[classInds])
print(np.sort(topUnits,axis=1))

# top n units with highest activation
print(activation_lastconv.shape)
acts = np.mean(activation_lastconv[0,:,:,:].reshape(weights_LR.shape[1],-1),axis=1)
topUnitsAct = np.argsort(acts)[-10:]
print(topUnitsAct)

for unit in topUnitsAct:#topUnits[0,-10:]:
    plt.imshow(activation_lastconv[0,unit,:,:])
    plt.colorbar()
    plt.show()

In [ ]:
[1010  965  732  836  548  794  986  530  802  979]
[1005  570 1004  605  732  742  530  986  979  802]
[687 742 979 726 732 986 802 570 530 626]
[313 538 445 802 409 986 820 969 535 687]


In [ ]:
folderPath = '../dataset/slamData/oxford/camTests/'
imageNames = os.listdir(folderPath)
print(imageNames)
camFeatsImgs = []
for imageName in imageNames:
    image = cv2.imread(folderPath+imageName)
    image = cv2.resize(image[:800,:,:], (256, 256))
    plt.imshow(image)
    plt.show()
    scores, activation_lastconv = getScoresAndLastConvAct(image)
    camFeats = getCAM(image,scores,activation_lastconv,showActs=False)
    camFeatsImgs.append(camFeats.flatten())

### Read data from a folder and store features for all images using one file per semantic category 

In [ ]:
oxDataFolders = ["2014-12-09-13-21-02/","2014-12-10-18-10-50/","2014-12-16-09-14-09/","2015-02-03-08-45-10/",
                "2015-05-19-14-06-38/"]
cameraTypes = ["mono_rear_rect/","stereo/left_rect/"]
resizeVals = [(512,512),(512,384)]
dataNum = 4
cameraNum = 1
resizeNum = 1
folderPath = '../dataset2/current/data/oxford/' + oxDataFolders[dataNum] + cameraTypes[cameraNum]
cameraTypes[1] = "stereo_left_rect/"

sampleIndsFile = "../dataset/slamData/oxford/samples/" + str(dataNum+1) + "-" + cameraTypes[cameraNum][:-1] + "-sampledInds.txt"

In [ ]:
print("Reading file - ", sampleIndsFile)
sampledInds = np.loadtxt(sampleIndsFile)
imageNameFmt = '{0:07d}'
print("Num Samples - ",sampledInds.shape)

In [ ]:
import time
camFeatsImgs = []
for i in range(sampledInds.shape[0]):
    index = int(sampledInds[i])
    imageName = folderPath + imageNameFmt.format(index) + ".png"
    image = cv2.imread(imageName)
    image = cv2.resize(image, resizeVals[resizeNum])[:320,:]
#     plt.imshow(image)
#     plt.show()
#     %time    
    scores, activation_lastconv = getScoresAndLastConvAct(image)
    time.sleep(0.1)
    camFeat = py_returnCAMmap(activation_lastconv, weights_LR)
    shp = camFeat.shape
    camFeatReshaped = camFeat.transpose([2,0,1]).reshape(shp[2],shp[0]*shp[1])
    camFeatsImgs.append(camFeatReshaped)
#     print(camFeatReshaped.shape)

    if i%20 == 0:
        print(i," images read.")
print(np.array(camFeatsImgs).shape)
featureArr = np.array(camFeatsImgs)

In [ ]:
dataStr = str(dataNum+1) + "-" + cameraTypes[cameraNum][:-1]
outFolderPath = "../dataset/slamData/oxford/labels-ConvCAM/" 
os.mkdir(outFolderPath+dataStr)
outFolderPath = outFolderPath + dataStr + "/"
for i in range(weights_LR.shape[0]):
    filePath = outFolderPath + str(i) + ".txt"
    np.savetxt(filePath,featureArr[:,i,:],fmt='%f')

### Cost Matrix
copied from semanticPlaceRec

In [ ]:
def getCostMatrix(probMat1,probMat2):
    
    newDiffMat = np.ndarray([probMat1.shape[0],probMat2.shape[0]])
    rowCounter = 0

    for a_row in probMat1:
#         newDiffMat[rowCounter,:] = np.dot(probMat2,a_row[:,...]) / np.square(probMat1.shape[1])
        newDiffMat[rowCounter,:] = (np.arccos(np.dot(probMat2,a_row[:,...]) / (np.linalg.norm(probMat2,axis=1)*
                                                                           np.linalg.norm(a_row))))/np.pi
#         newDiffMat[rowCounter,:] = np.linalg.norm(probMat2-a_row[:,...],axis=1)
#         newDiffMat[rowCounter,:] = np.sum(np.divide(abs(probMat2-a_row[:,...]),probMat2),axis=1)
        
        rowCounter+=1
    assert(rowCounter==probMat1.shape[0])
    
    return newDiffMat

In [ ]:
camFeatsImgs2 = (camFeatsImgs - np.mean(camFeatsImgs,axis=0))/np.var(camFeatsImgs,axis=0)

In [ ]:
# print(np.mean(act_pool1),np.mean(act_pool2))
# print(np.argsort(act_pool1[92,:])[-10:],np.argsort(act_pool2[92,:])[-10:])
diff = getCostMatrix(np.array(camFeatsImgs2),np.array(camFeatsImgs2))
# print(diff)
plt.figure(figsize=(10,10))
plt.imshow(diff,'hot')
plt.colorbar()
plt.show()
# plt.imshow(act_pool1[:,500:600])
# plt.colorbar()